needs `optimum`, `auto-gptq` (--extra-index-url https://huggingface.github.io/autogptq-index/whl/cu115/), `transformers -U`

In [ ]:
import torch
torch.cuda.get_device_properties(0) 

In [ ]:
from transformers import AutoModelForCausalLM

# model_name = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             torch_dtype=torch.float16, device_map="auto")

In [ ]:
model.cuda()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

prompt = "Tell me about O'Reilly Safari"
prompt_template=f'''[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt}[/INST]

'''

In [ ]:
%%time
print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

In [ ]:
!nvidia-smi